In [1]:
import pandas as pd

from datetime import date

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', 1000)

In [3]:
df_Xcite = pd.read_excel('Product_Catalog_Outputs\Xcite_product_catalog_2023_07_05.xlsx')
df_Zain = pd.read_excel('Product_Catalog_Outputs\Zain_product_catalog_2023_07_05.xlsx')

df_Xcite['name_split'] = df_Xcite['product_name'].str.lower().str.split(" ").apply(lambda x: set(x))
df_Zain['name_split'] = df_Zain['product_name'].str.lower().str.split(" ").apply(lambda x: set(x))

df_Xcite['price_float'] = df_Xcite['product_price'].str.split(' ').str[0].str.replace(',', '').astype(float)
df_Zain['price_float'] = df_Zain['product_price_cash'].str.split(' ').str[1].str.replace(',', '').astype(float)

Unique_ID_fields_Zain = df_Zain.loc[:,['name_split', 'product_brand', 'price_float']]
Unique_ID_fields_Xcite = df_Xcite.loc[:,['name_split', 'product_brand', 'price_float', 'in_out_stock']]

In [16]:
# TUNE PARAMETERS

intersection_threshold = 0.66
price_floor   = 0.7 
price_ceiling = 1.3

In [17]:
matching = [
    # match list
    [' '.join(list(name_list_zain)), ' '.join(list(name_list_xcite)), Unique_ID_fields_Zain['price_float'][index_zain], Unique_ID_fields_Xcite['price_float'][index_xcite]]
    
    # name_split iteration
    for index_zain, name_list_zain in enumerate(Unique_ID_fields_Zain['name_split'])
    for index_xcite, name_list_xcite in enumerate(Unique_ID_fields_Xcite['name_split'])
    
    # condition filters (at least 70% intersection, matching brand name, within 50% +/- price agreement, 'in stock')
    if name_list_zain.intersection(name_list_xcite) and len(name_list_zain.intersection(name_list_xcite))/len(name_list_zain) >= intersection_threshold
    and str(Unique_ID_fields_Zain['product_brand'][index_zain]).lower() == str(Unique_ID_fields_Xcite['product_brand'][index_xcite]).lower()
    and Unique_ID_fields_Zain['price_float'][index_zain] >= Unique_ID_fields_Xcite['price_float'][index_xcite] * price_floor
    and Unique_ID_fields_Zain['price_float'][index_zain] <= Unique_ID_fields_Xcite['price_float'][index_xcite] * price_ceiling
    and Unique_ID_fields_Xcite['in_out_stock'][index_xcite] == 'in_stock'
    # ADD ZAIN IN_OUT_STOCK
    # ENRICH ZAIN NAMING CONVENTIONS AND DATA SET
]

In [18]:
percent_intersection_list = [
    round((len(name_list_zain.intersection(name_list_xcite)) / len(name_list_zain))*100,2)
    for index_zain, name_list_zain in enumerate(Unique_ID_fields_Zain['name_split'])
    for index_xcite, name_list_xcite in enumerate(Unique_ID_fields_Xcite['name_split'])
    if (
        len(name_list_zain.intersection(name_list_xcite)) / len(name_list_zain) >= intersection_threshold
        and str(Unique_ID_fields_Zain['product_brand'][index_zain]).lower() == str(Unique_ID_fields_Xcite['product_brand'][index_xcite]).lower()
        and Unique_ID_fields_Zain['price_float'][index_zain] >= Unique_ID_fields_Xcite['price_float'][index_xcite] * price_floor
        and Unique_ID_fields_Zain['price_float'][index_zain] <= Unique_ID_fields_Xcite['price_float'][index_xcite] * price_ceiling
        and Unique_ID_fields_Xcite['in_out_stock'][index_xcite] == 'in_stock'
    )
]

In [19]:
match_df = pd.DataFrame(matching)

In [20]:
match_df.columns = ['zain_product', 'xcite_product', 'zain_price', 'xcite_price']

In [21]:
 match_df['price_difference [%]'] = round(((match_df['zain_price'] - match_df['xcite_price'])/match_df['zain_price'])*100,2)

In [23]:
match_df['percent_intersection [%]'] = percent_intersection_list

In [24]:
 match_df.sort_values('price_difference [%]')

,zain_product,xcite_product,zain_price,xcite_price,price_difference [%],percent_intersection [%]
249,4 laptop surface,"i7 laptop core intel ssd, 14-inch studio 16gb - microsoft gen, 512gb convertible 11th platinum ram, surface",420.00,599.90,-42.83,66.67
243,5g galaxy m33,phone samsung s21 256gb grey galaxy 5g fe -,82.95,117.90,-42.13,66.67
241,5g galaxy a33,phone samsung s21 256gb galaxy 5g fe - olive,84.95,119.90,-41.14,66.67
240,5g galaxy a33,phone samsung s21 256gb galaxy 5g fe - lavender,84.95,119.90,-41.14,66.67
248,go laptop surface,"laptop ssd, sq2, 13-inch 16gb 256gb - microsoft x platinum ram, surface pro",249.00,349.90,-40.52,66.67
22,8 watch series,gps watch cellular red case with band regular + sport aluminium 45mm series 8 apple -,121.00,169.90,-40.41,100.00
229,iphone 12 pro,gold - 5g 14 apple 1tb iphone pro,341.50,474.50,-38.95,66.67
234,iphone 12 pro,space - 5g black 14 apple 1tb iphone pro,341.50,474.50,-38.95,66.67
232,iphone 12 pro,- 5g 14 silver apple 1tb iphone pro,341.50,474.50,-38.95,66.67
242,5g galaxy a33,phone samsung s21 256gb grey galaxy 5g fe -,84.95,117.90,-38.79,66.67


In [ ]:
match_df.to_excel(f'Analytics\matches_{date.today().strftime("%Y_%m_%d")}.xlsx')